In [1]:
import multiprocessing as mp
import warnings
import pandas as pd

In [2]:
warnings.filterwarnings('ignore')

In [3]:
f = open('datasets/guesses.txt','r')
guesses = []

for line in f:
    guesses.append(line.strip())

print(str(len(guesses)))

12972


In [4]:
f = open('datasets/solutions.txt','r')
solutions = []

for line in f:
    solutions.append(line.strip())

print(str(len(solutions)))

2315


In [5]:
words5 = guesses

In [6]:
# this should be an empty list, checking to be sure we do not have non alpha chars in the words
print([i for i in words5 if not i.isalpha()])

[]


In [7]:
len(words5)

12972

In [8]:
# word = 'wifie'
word_doubles = []
for word in words5:
    word_counts = 0
    for letter in word:
        # print(word)
        # print(letter)
        # print(str(word.count(letter)))
        word_counts += word.count(letter)
    if word_counts > 5:
        word_doubles.append(word)

In [9]:
len(word_doubles)

4650

In [10]:
df_words = pd.DataFrame(words5)
df_words.columns=['words']

df_doubles = pd.DataFrame(word_doubles)
df_doubles.columns=['word']

In [11]:
df_doubles['double']=True

In [12]:
df_doubles.head()

,word,double
0,sissy,True
1,humph,True
2,awake,True
3,evade,True
4,naval,True


In [13]:
print(df_doubles.head())
index = df_doubles.index
number_of_rows = len(index)
print(number_of_rows)
df_doubles.to_csv('datasets/double_letters.csv')

    word  double
0  sissy    True
1  humph    True
2  awake    True
3  evade    True
4  naval    True
4650


In [14]:
df_words['first'] = df_words['words'].str.slice(0,1)
df_words['second'] = df_words['words'].str.slice(1,2)
df_words['third'] = df_words['words'].str.slice(2,3)
df_words['fourth'] = df_words['words'].str.slice(3,4)
df_words['fifth'] = df_words['words'].str.slice(4,5)

In [15]:
df_words.head()

,words,first,second,third,fourth,fifth
0,cigar,c,i,g,a,r
1,rebut,r,e,b,u,t
2,sissy,s,i,s,s,y
3,humph,h,u,m,p,h
4,awake,a,w,a,k,e


In [16]:
df_words['first'][0]==df_words['second'][0]

False

In [17]:
alphas = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [18]:
dfalpha = pd.DataFrame(alphas)
dfalpha.columns = ['letters']
dfalpha.set_index('letters', inplace=True)

In [19]:
# build the frequerncy table from each smaller dataframe
df_counts = df_words['first'].value_counts()
df_counts_second = df_words['second'].value_counts()
df_counts_third = df_words['third'].value_counts()
df_counts_fourth = df_words['fourth'].value_counts()
df_counts_fifth = df_words['fifth'].value_counts()

dfranks = pd.concat([dfalpha, df_counts], axis=1)
dfranks = pd.concat([dfranks, df_counts_second], axis=1)
dfranks = pd.concat([dfranks, df_counts_third], axis=1)
dfranks = pd.concat([dfranks, df_counts_fourth], axis=1)
dfranks = pd.concat([dfranks, df_counts_fifth], axis=1)

In [20]:
df_counts.head()

s    1565
c     922
b     909
p     859
t     815
Name: first, dtype: int64

In [21]:
dfranks.head()

,first,second,third,fourth,fifth
a,737,2263,1236,1074,680
b,909,81,335,243,59
c,922,176,392,411,127
d,685,84,390,471,823
e,303,1628,882,2327,1522


In [22]:
dfranks['first']['a']

737

In [23]:
def freq_ranking(word):
    val = dfranks['first'][word[0]] + dfranks['second'][word[1]] +dfranks['third'][word[2]] + dfranks['fourth'][word[3]] +dfranks['fifth'][word[4]]
    return val

In [24]:
df_word_ranks = pd.DataFrame({'word' : [],'value':[]})
word_freq = {'word':'value'}
for word in words5:
    df_word_ranks.loc[len(df_word_ranks.index), 'word'] = word
    df_word_ranks.loc[len(df_word_ranks.index)-1, 'value'] = freq_ranking(word)

In [25]:
df_word_ranks.head()

,word,value
0,cigar,4416.0
1,rebut,3719.0
2,sissy,5298.0
3,humph,2975.0
4,awake,4161.0


In [26]:
df_word_ranks = df_word_ranks.merge(df_doubles,how='left')
df_ranks_no_doubles = df_word_ranks.loc[df_word_ranks['double'] != True]

In [27]:
df_word_ranks.to_csv('datasets/word_ranks.csv')

In [28]:
df_word_ranks.head()

,word,value,double
0,cigar,4416.0,NaN
1,rebut,3719.0,NaN
2,sissy,5298.0,True
3,humph,2975.0,True
4,awake,4161.0,True


In [29]:
print('with doubles')
print (df_word_ranks['word'][df_word_ranks['value'].idxmax()])
print (df_word_ranks['value'][df_word_ranks['value'].idxmax()])
print('no double letters')
print (df_ranks_no_doubles['word'][df_ranks_no_doubles['value'].idxmax()])
print (df_ranks_no_doubles['value'][df_ranks_no_doubles['value'].idxmax()])

with doubles
sores
11144.0
no double letters
cares
10668.0


In [30]:
# here we determine the next best word based on results of the best word per wordle feedback
# how to determine which items were elimiated
#  ~ before a contains statement makes it a "not" I was lucky to find this
display(df_word_ranks.loc[(~df_word_ranks['word'].str.contains('t'))])

,word,value,double
0,cigar,4416.0,NaN
2,sissy,5298.0,True
3,humph,2975.0,True
4,awake,4161.0,True
5,blush,3161.0,NaN
...,...,...,...
12967,zuzim,2496.0,True
12968,zygal,2290.0,NaN
12969,zygon,1968.0,NaN
12970,zymes,7172.0,NaN


In [31]:
grey = ['s','l','a','t','e']

In [32]:
df_sub = df_word_ranks
for l in grey:
    # print(l)
    # print(str(len(df_sub.index)))
    df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
df_sub

,word,value,double
3,humph,2975.0,True
48,mimic,3594.0,True
49,pound,5233.0,NaN
54,booby,5542.0,True
59,ivory,3230.0,NaN
...,...,...,...
12955,zoppo,3372.0,True
12958,zorro,4507.0,True
12967,zuzim,2496.0,True
12969,zygon,1968.0,NaN


In [33]:
print (df_sub['word'][df_sub['value'].idxmax()])
print (df_sub['value'][df_sub['value'].idxmax()])

corny
6305.0


In [34]:
test = "dogg"
print(test.count('g'))

2


In [35]:
word = 'score'
guess = 'scres'
score = 0

for i in range(0,4):
    # find character match in locations
    if guess[i]==word[i]:
        print(f'exact char match on letter {letter} in positions {i}')
        score += 2

exact char match on letter c in positions 0
exact char match on letter c in positions 1


In [36]:
solution = ['write','fight','soyet','cleat']

df_geuss_value = pd.DataFrame({'guess' : [],'score':[], 'answer':[]})

for idx, word in enumerate(solution):
    print(str(idx))
    print(str(word))

    df_geuss_value.loc[idx, 'guess'] = guess
    df_geuss_value.loc[idx, 'score'] = score
    df_geuss_value.loc[idx, 'answer'] = word

df_geuss_value

0
write
1
fight
2
soyet
3
cleat


,guess,score,answer
0,scres,4.0,write
1,scres,4.0,fight
2,scres,4.0,soyet
3,scres,4.0,cleat


In [37]:
df_geuss_value.head()

,guess,score,answer
0,scres,4.0,write
1,scres,4.0,fight
2,scres,4.0,soyet
3,scres,4.0,cleat


In [38]:
df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})

In [39]:
# turn my existing loop into a function that can be parallelized
def word_val(guess, solution_list):
    df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})


# make a smaller dataframe for each guess and have that be appended to the larger dataframe for each individual guess


    for idx, word in enumerate(solution_list):
        results = []
        score = 0
        score_exact = 0
        # print(word)
        for letter in guess:
    # match of letter
    # give one point
            # print(f' for the guess {guess}, and the answer {word} the score for letter: {letter} is {word.count(letter)}')
            score += int(word.count(letter))
        if word.count(letter) >=1:
            for i in range(0,4):
    # find character match in locations
                char = guess[i]
                if guess[i]==word[i]:
                    # print(f'exact char match on letter {char} in positions {i}')
                    score_exact += 1
        
        df_geuss_value.loc[idx, 'guess'] = guess
        df_geuss_value.loc[idx, 'score_match'] = score
        df_geuss_value.loc[idx, 'answer'] = word
        df_geuss_value.loc[idx, 'score_exact_match'] = score_exact

    # results.append(guess)
    # results.append(score)
    # results.append(word)
    # results.append(score_exact)
    # results.append(idx)

    return df_geuss_value



In [40]:
def row_results(results_df):
    # global df_geuss_value
    # df_geuss_value.loc[idx, 'guess'] = results[0]
    # df_geuss_value.loc[idx, 'score_match'] = results[1]
    # df_geuss_value.loc[idx, 'answer'] = results[2]
    # df_geuss_value.loc[idx, 'score_exact_match'] = results[3]
    global df_geuss_value
    df_geuss_value = df_geuss_value.append(results_df)
    # df_geuss_value = pd.concat(results_df)


In [41]:
few_guesses = guesses[0:50]
# few_guesses

In [42]:
# warnings.filterwarnings(action='once')

# for idx, guess in enumerate(few_guesses):
#     # print(guess)
#     # print(word_val(guess,solutions))
#     row_results(word_val(guess,solutions))

In [43]:
print(df_geuss_value.head())
index = df_geuss_value.index
number_of_rows = len(index)
print(number_of_rows)
df_geuss_value.to_csv('datasets/guess_value_iterations_guesses')

Empty DataFrame
Columns: [guess, score_match, answer, score_exact_match]
Index: []
0


In [44]:
df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})

In [45]:
print(f'Number of CPUs availible: {mp.cpu_count()}')
pool_count = mp.cpu_count() -2
pool = mp.Pool(pool_count)
# pool = mp.Pool(8) # if you would like to use less of your PC's total threads just specify a number less than your total here 

Number of CPUs availible: 16


In [54]:
# if you plan to run this, it will take a while, this took 46 minutes over 14 threads the last time I ran it.

for guess in guesses:
    pool.apply_async(word_val, args=(guess,solutions),callback=row_results)

pool.close()
pool.join()

In [55]:
print(df_geuss_value.head())
index = df_geuss_value.index
number_of_rows = len(index)
print(number_of_rows)
df_geuss_value.to_csv('datasets/guess_value_iterations_guesses.csv') # this file will be too large for github and I've not included it

   guess  score_match answer  score_exact_match
0  cigar          5.0  cigar                4.0
1  cigar          1.0  rebut                0.0
2  cigar          1.0  sissy                0.0
3  cigar          0.0  humph                0.0
4  cigar          2.0  awake                0.0
30030180
